In [2]:
import numpy as np 
import pandas as pd
import allel
import bokeh
import bokeh.plotting
import malariagen_data
%matplotlib inline
import sys
%run hapclust.py
bokeh.io.reset_output()

### Load pops

Which populations do we want to look at?

In [3]:
cohorts = [
    # Ag1000G phase 3 sample sets in Ag3.0
    "AG1000G-GH", 
    'AG1000G-ML-A',
     'AG1000G-BF-A',
     'AG1000G-BF-B',
     'AG1000G-GN-A',
     'AG1000G-GN-B',
    'AG1000G-TZ',
    # Amenta-Etego sample sets in Ag3.3
    # GAARDIAN sample set in Ag3.4
    '1244-VO-GH-YAWSON-VMF00149',
    # GAARD Ghana sample set in Ag3.2
     "1244-VO-GH-YAWSON-VMF00051",
     '1245-VO-CI-CONSTANT-VMF00054',
     '1253-VO-TG-DJOGBENOU-VMF00052',
     '1237-VO-BJ-DJOGBENOU-VMF00050'
]

name = 'coeae1f'
contig= '2L'
coeregion = '2L:28530000-28570000'

transcript= "AGAP006227-RA"
gene = transcript.replace("-RA", "")

locstart = 28_000_000
locstop = 29_000_000

coeae1f_start = 28_545_396
coeae2f_start = 28_548_433

In [4]:
ag3 = malariagen_data.Ag3(pre=True)
metadata = ag3.sample_metadata(cohorts)

Load sample metadata:   0%|          | 0/12 [00:00<?, ?it/s]

Define cohort queries

In [5]:
wa_countries = ['Ghana', 'Burkina Faso', 'Mali', 'Guinea', 'Togo', 'Benin', "Cote d'Ivoire"]
ea_countries = 'Tanzania'

early_years = [2012, 2013, 2014]
late_years = [2017, 2018]

sample_queries = {'WA-gambiae-2012-14': 'country in ["Ghana", "Burkina Faso", "Mali", "Guinea", "Togo", "Benin", "Cote d\'Ivoire"] & year in [2012, 2013, 2014] & taxon == "gambiae"',
                 'WA-gambiae-2017-18':  'country in ["Ghana", "Burkina Faso", "Mali", "Guinea", "Togo", "Benin", "Cote d\'Ivoire"] & year in [2017, 2018] & taxon == "gambiae"',
                 'WA-coluzzii-2012-14': 'country in ["Ghana", "Burkina Faso", "Mali", "Guinea", "Togo", "Benin", "Cote d\'Ivoire"] & year in [2012, 2013, 2014] & taxon == "coluzzii"',
                 'WA-coluzzii-2017-18': 'country in ["Ghana", "Burkina Faso", "Mali", "Guinea", "Togo", "Benin", "Cote d\'Ivoire"] & year in [2017, 2018] & taxon == "coluzzii"',
                 'EA-gambiae-2013-15':  'country in "Tanzania" & taxon == "gambiae"',
                 'EA-arabiensis-2012-15': 'country == "Tanzania" & taxon == "arabiensis"'}

#### Run across genome and plot H123 together on one plot 

In [6]:
def garuds_h(contig, cohorts, query_name, sample_query, size=1200, step=600, save=False):

    snps = ag3.haplotypes(region=contig, sample_sets=cohorts, sample_query=sample_query, analysis='gamb_colu_arab')
  #  meta = ag3.sample_metadata(sample_sets=cohort, sample_query)
    
    print(f"-------------------------------- {query_name} H123 -----------------------------------------")    

    haps = allel.GenotypeArray(snps['call_genotype'])
    haps = haps.take(np.random.choice(haps.shape[1], 100), axis=1)
    print(haps.shape)
    probe.log("Subsetting to segregating sites")
    seg = haps.count_alleles().is_segregating()
    print(f"There are {seg.sum()} segregating sites")
    haps = haps.compress(seg, axis=0).to_haplotypes()
    probe.log("Running H123...")
    h1, h12, h123, h2_1 = allel.moving_garud_h(haps, size=size, step=step)
    midpoints = allel.moving_statistic(snps['variant_position'].values[seg], np.median, size=size, step=step)   
    assert midpoints.shape == h12.shape, f"arrays not same shape, midpoints shape - {midpoints.shape}, value shape - {h123.shape}"
    
    pd.DataFrame([h123, midpoints]).to_csv(f"../../results/{query_name}.h123.tsv",sep="\t")
    return(h123, midpoints)

In [8]:
sample_queries['EA-arabiensis-2012-15']

'country == "Tanzania" & taxon == "arabiensis"'

In [10]:
import probe as probe

In [11]:
h123_dict[name], midp_dict[name] = garuds_h(contig, cohorts, query_name='EA-arabiensis-2012-15', sample_query=sample_queries['EA-arabiensis-2012-15'], size=1200, step=600)

-------------------------------- EA-arabiensis-2012-15 H123 -----------------------------------------
(13431083, 100, 2)
Subsetting to segregating sites
There are 916244 segregating sites
Running H123...


NameError: name 'h123_dict' is not defined

In [35]:
h123_dict = {}
midp_dict = {}

for name, query in sample_queries.items():
    
    h123_dict[name], midp_dict[name] = garuds_h(contig, cohorts, query_name=name, sample_query=query, size=1200, step=600)

-------------------------------- WA_gambiae_2012-14 H123 -----------------------------------------
(13431083, 100, 2)
Subsetting to segregating sites
There are 2669804 segregating sites
Running H123...
-------------------------------- WA_gambiae_2017-18 H123 -----------------------------------------
(13431083, 100, 2)
Subsetting to segregating sites
There are 1979699 segregating sites
Running H123...
-------------------------------- WA_coluzzii_2012-14 H123 -----------------------------------------
(13431083, 100, 2)
Subsetting to segregating sites
There are 2430988 segregating sites
Running H123...
-------------------------------- WA_coluzzii_2017-18 H123 -----------------------------------------
(13431083, 100, 2)
Subsetting to segregating sites
There are 1711321 segregating sites
Running H123...
-------------------------------- EA_gambiae_2013-15 H123 -----------------------------------------
(13431083, 100, 2)
Subsetting to segregating sites
There are 1542865 segregating sites
Runn

SyntaxError: EOL while scanning string literal (<unknown>, line 1)